In [22]:
import pandas as pd
import numpy as np

In [57]:
data_path = '../data/'
df = pd.read_csv(data_path + 'ner_dataset.zip', encoding="latin1")

In [58]:
df.head().T

,0,1,2,3,4
Sentence #,Sentence: 1,NaN,NaN,NaN,NaN
Word,Thousands,of,demonstrators,have,marched
POS,NNS,IN,NNS,VBP,VBN
Tag,O,O,O,O,O


In [65]:
sent_idx = df[df['Sentence #'].notna()].index.values

In [123]:
class NERSentence:
    def __init__(self, df, idx_start=0, indicator_var='Sentence #', word_var='Word'):
        self.df = df
        self.i = idx_start
        self.word_var = word_var
        
        self.idx_all = df[df[indicator_var].notna()].index.values
        self.idx_a = self.idx_all[self.i]
        self.idx_b = self.idx_all[1]
        self.i += 1
        
    def update_sentence_range(self):
        self.i += 1
        self.idx_a = self.idx_b
        self.idx_b = self.idx_all[self.i]
                
    
    def get_next_sentence(self):
        s = ''
        for i in range(self.idx_a, self.idx_b):
            s = s + ' ' + df[i:i+1][self.word_var].values[0]
            
        self.update_sentence_range()
    
        return s


In [127]:
se = NERSentence(df)
se.get_next_sentence()

' Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [128]:
se.get_next_sentence()

' Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "'

In [130]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load the NER tagger
tagger = SequenceTagger.load('ner')

se = NERSentence(df)

for i in range(3):
    # run NER over sentence
    s_next = se.get_next_sentence()
    s = Sentence(s_next)
    tagger.predict(s)
    print('------------\n'+s_next+'\n')
    for entity in s.get_spans('ner'):
        print(entity)

2022-06-22 15:56:07,111 loading file /Users/jason/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-22 15:56:08,444 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
------------
 Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

Span[6:7]: "London" → LOC (0.9999)
Span[12:13]: "Iraq" → LOC (1.0)
Span[18:19]: "British" → MISC (0.9976)
------------
 Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "

Span[18:22]: "Bush Number One Terrorist" → MISC (0.6677)
Span[25:28]: "Stop the Bombings" → MISC (0.963)
------------
 They marched